In [1]:
import IPython.display as ipd

import numpy as np
import torch

from audio_processing import griffin_lim

from scipy.io.wavfile import read
import time
from reconstruct_functions import *
import random
import os

def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")

set_seed(114514)

Random seed set as 114514


In [2]:
hop_length = 3
win_length = 12 ###at least 4*hop_length, even number
full_length = win_length + 4*hop_length

def get_Yw(input_signal, window, hop_length =1, win_length=4):
    # input shape: [1,T]
    # output shape: [(T+N-1)/H, T/2+1]
    T = input_signal.shape[-1]
    H = hop_length
    N = win_length
    Yw_real = torch.zeros(((T+N-1)//H, T//2+1), dtype=torch.float64)
    Yw_imag = torch.zeros(((T+N-1)//H, T//2+1), dtype=torch.float64)
    # pi = torch.acos(torch.zeros(1, dtype=torch.float64)).item() * 2
    for m in range((T+N-1)//H):
        for n in range(T//2+1):
            for t in range(T):
                if m*H-t<0 or m*H-t>=N:
                    continue
                Yw_real[m,n] = Yw_real[m,n]+ input_signal[0,t]*window[m*H-t]*np.cos(-2*np.pi*n*t/T)
                Yw_imag[m,n] = Yw_imag[m,n]+ input_signal[0,t]*window[m*H-t]*np.sin(-2*np.pi*n*t/T)
    return torch.sqrt(Yw_real**2 + Yw_imag**2)

In [3]:


def compute_reconstruction(input_signal, window):
    # shape: [1,:]
    magnitude  = get_Yw(input_signal, window, hop_length, win_length)
    
    ans = reconstruct_from_Yw_with_H_3(magnitude, window, hop_length)
    
    if ans[0,0]*input_signal[0,0]<0:
        ans[:,:] = -ans[:,:]
    print('ans', ans)
    
    return ans


In [4]:
test_time = 5
data_arr = []

for idx in range(test_time):
    print('######', idx)
    audio_origin = torch.rand((1,full_length), dtype=torch.float64)*2-1
    
    window = torch.rand((win_length), dtype=torch.float64)
    print('signal', audio_origin)
    print('window', window)
    ans = compute_reconstruction(audio_origin, window)
        
    data_arr.append(torch.mean(torch.abs(audio_origin[0,:] - ans[0,:])))
    print(torch.mean(torch.abs(audio_origin[0,:] - ans[0,:])))

print(np.mean(data_arr))
print(np.var(data_arr))

###### 0
signal tensor([[ 0.0481,  0.6044,  0.4005,  0.3006,  0.1836, -0.8655,  0.7392, -0.5941,
          0.0399,  0.1515,  0.5950, -0.8755, -0.2879, -0.1142,  0.7590,  0.0166,
         -0.5702, -0.4450, -0.0190, -0.6339,  0.7961,  0.1058, -0.3611, -0.4709]],
       dtype=torch.float64)
window tensor([0.3657, 0.4342, 0.2511, 0.8519, 0.3015, 0.4464, 0.0398, 0.2660, 0.3985,
        0.4288, 0.2968, 0.2912], dtype=torch.float64)


100%|██████████| 6/6 [00:00<00:00, 2017.30it/s]


ans [[ 0.04805358  0.60435092  0.40050719  0.300559    0.18361471 -0.86551264
   0.73915877 -0.59405125  0.03991664  0.15152053  0.59503475 -0.87545772
  -0.28792413 -0.11418192  0.7589735   0.01663292 -0.5702101  -0.44499301
  -0.01900145 -0.63386279  0.79608359  0.10575303 -0.36105869 -0.47088567]]
tensor(8.5565e-14, dtype=torch.float64)
###### 1
signal tensor([[ 0.2368, -0.5671,  0.2125,  0.4021, -0.5473,  0.3924, -0.1698,  0.6180,
         -0.8492,  0.0102,  0.8805, -0.4135,  0.2323,  0.1480,  0.6167, -0.4830,
         -0.0279,  0.9758, -0.3849,  0.1666, -0.9009, -0.5043,  0.4210,  0.4831]],
       dtype=torch.float64)
window tensor([0.4221, 0.1473, 0.2368, 0.4079, 0.5871, 0.1187, 0.5853, 0.6486, 0.6264,
        0.7310, 0.4809, 0.8059], dtype=torch.float64)


100%|██████████| 6/6 [00:00<00:00, 2117.09it/s]


ans [[ 0.23683362 -0.56706514  0.21251349  0.40205853 -0.54726437  0.39241571
  -0.16983447  0.61804083 -0.8491509   0.01022333  0.88050382 -0.41347834
   0.23225625  0.14802665  0.61666715 -0.4829868  -0.02792985  0.97577104
  -0.38487804  0.16657397 -0.90094731 -0.50433919  0.42097012  0.48313482]]
tensor(2.5433e-13, dtype=torch.float64)
###### 2
signal tensor([[ 8.6909e-01,  1.8586e-01,  8.1710e-01, -6.3226e-01, -4.7155e-01,
         -9.5325e-01, -4.4459e-01, -5.0644e-01,  5.4514e-01, -7.9202e-01,
         -6.6252e-01, -3.1327e-01, -6.0484e-01, -8.3696e-01,  5.1339e-01,
         -5.0494e-01,  1.2453e-01,  4.7020e-01,  4.7266e-01, -2.9730e-01,
          5.4404e-01, -6.3159e-01,  3.2465e-04,  6.0191e-01]],
       dtype=torch.float64)
window tensor([0.6654, 0.5426, 0.1557, 0.7991, 0.7608, 0.1754, 0.2026, 0.4013, 0.2939,
        0.6873, 0.4220, 0.7105], dtype=torch.float64)


...

ans [[ 8.69087076e-01  1.85858536e-01  8.17103245e-01 -6.32259334e-01
  -4.71554195e-01 -9.53250273e-01 -4.44592245e-01 -5.06440990e-01
   5.45143183e-01 -7.92015396e-01 -6.62522854e-01 -3.13272633e-01
  -6.04837748e-01 -8.36962587e-01  5.13389697e-01 -5.04941916e-01
   1.24532326e-01  4.70204470e-01  4.72656895e-01 -2.97300339e-01
   5.44042716e-01 -6.31585423e-01  3.24650767e-04  6.01913216e-01]]
tensor(1.7311e-12, dtype=torch.float64)
###### 3
signal tensor([[ 0.3695, -0.8292,  0.9244, -0.1416, -0.1708, -0.1730,  0.8796,  0.3604,
         -0.5419,  0.3376,  0.9650, -0.9202, -0.2147, -0.5329, -0.3018, -0.4644,
         -0.0391, -0.4169,  0.6877, -0.3169,  0.4257,  0.1995,  0.4204,  0.3825]],
       dtype=torch.float64)
window tensor([0.8777, 0.4153, 0.4426, 0.4991, 0.1047, 0.0734, 0.8764, 0.4606, 0.7445,
        0.1401, 0.4872, 0.4990], dtype=torch.float64)



100%|██████████| 6/6 [00:00<00:00, 2170.96it/s]


ans [[ 0.36949566 -0.82918551  0.92439093 -0.1416182  -0.17083722 -0.17300594
   0.87961974  0.36036377 -0.54189285  0.33755741  0.96501251 -0.92021382
  -0.21467471 -0.53289109 -0.30176181 -0.46444415 -0.0391089  -0.4168767
   0.6876735  -0.31685244  0.42574138  0.19949462  0.42037042  0.3825448 ]]
tensor(1.1401e-13, dtype=torch.float64)
###### 4
signal tensor([[-0.1677, -0.7570,  0.2763, -0.4136, -0.9374,  0.4671, -0.4160,  0.1631,
          0.0493, -0.7231, -0.3512,  0.1391,  0.0035,  0.5673,  0.3027,  0.3263,
          0.6122,  0.8561,  0.5883,  0.1585, -0.4953, -0.2611,  0.0256,  0.8444]],
       dtype=torch.float64)
window tensor([0.7643, 0.6923, 0.2478, 0.8126, 0.4063, 0.8271, 0.0207, 0.1154, 0.5783,
        0.2318, 0.0118, 0.4411], dtype=torch.float64)


100%|██████████| 6/6 [00:00<00:00, 2023.46it/s]

ans [[-0.16767564 -0.7570356   0.27625407 -0.413572   -0.9374164   0.4670854
  -0.41600202  0.16314604  0.04926667 -0.72314992 -0.35116966  0.13906474
   0.00353267  0.5673161   0.3027352   0.32629265  0.61219725  0.85610014
   0.58832341  0.15851554 -0.49532776 -0.26108691  0.02563098  0.8444091 ]]
tensor(1.0689e-13, dtype=torch.float64)
4.583755914438527e-13
4.0850972079645985e-25
